In [1]:
import time
import pandas as pd
from datetime import datetime

import bernielib as bl

In [2]:
ber = bl.robot()

In [3]:
ber.home()

# Uptaking small amount of neede sample (10-100 uL)

The goal is to retrieve all the liquid

## Specifying parameters

In [4]:
# Will refill from here
water = bl.createSample('25ml', 'water', ber.reagents_rack, 0, 0, 25000)
# Will dump to here
waste = bl.createSample('25ml', 'liquid_waste', ber.reagents_rack, 0, 1, 0)
# Sample to uptake from. Starts empty
sample1 = bl.createSample('eppendorf', 's1', ber.samples_rack, 1, 0, 0)

In [49]:
# Maximum plunger movement
ber.setPlungerMaxCoord(35)
ber.setLiquidUptakeLowVolBottomOffset(0.2)
#ber.tips_rack.refill()

## Preparing the tube

In [5]:
ber.dumpTipToWaste()

In [50]:
# Removing all possible liquid from the previous (failed?) attempts
ber.pickUpNextTip()
ber.move(z=50)
ber.uptakeLiquid(sample1, 200)
ber.move(z=50)
ber.dispenseLiquid(waste, 200, blow_extra=True)
ber.move(z=50)
ber.dumpTipToWaste()

In [51]:
sample1.getZBottom()

136.125

In [26]:
ber.pickUpNextTip()

(0, 9)

In [6]:
ber.move(z=50)

In [10]:
ber.moveToSample(sample1)

In [29]:
ber.move(z=sample1.getZBottom())

In [30]:
ber.moveDownUntilPress(step=0.1, threshold=100)

136.425

In [16]:
ber.uptakeLiquid(sample1, 200, ignore_bottom_z_calibration=True)

In [13]:
sample1.getZBottom()

136.225

In [16]:
ber.move(z=136)

In [17]:
ber.moveDownUntilPress(step=0.1, threshold=100)

136.3

In [90]:
# Putting liquid into the tube
ber.pickUpNextTip()
ber.move(z=50)
ber.uptakeLiquid(water, 30)
ber.move(z=50)
ber.dispenseLiquid(sample1, 30, blow_extra=True, plunger_retract=False)
ber.touchWall(sample1)
ber.moveAxis(axis='A', dist=0)
ber.move(z=50)
ber.dumpTipToWaste()

## Actual uptake

In [5]:
def getCloseToBottomZ(sample, tip_length_compensation):
    approximate_volume = sample.getCloseToBottomVol()
    approximate_z = sample.calcAbsLiquidLevelFromVol(approximate_volume, added_length=tip_length_compensation)
    return approximate_z

In [6]:
def stepOfACircle(robot, axis, radius, delay, dZ, new_vol):
    robot.moveAxisDelta(axis, radius)
    robot.moveAxisDelta('Z', dZ)
    robot.movePipetteToVolume(new_vol)
    time.sleep(delay)
    robot.moveAxisDelta('Z', -dZ)
    time.sleep(delay)
    robot.moveAxisDelta(axis, -radius)

In [7]:
def uptakeAllLiquid(robot, sample, extra_vol=50, z_safe=50, ignore_calibration=True, R=2, T=0.5):
    
    
    x, y = sample.getCenterXY()
    tip_length_compensation = robot._calcExtraLength()
    z_near_bottom = getCloseToBottomZ(sample, tip_length_compensation)
    robot.move(z=z_safe)
    robot.moveToSample(sample=sample)
    
    # Setting the plunger
    volume = sample.getVolume()
    volume_to_uptake = volume + extra_vol
    if volume_to_uptake > 250:
        # Robot will be unable to uptake all liquid at this point
        return False
    
    robot.movePipetteToVolume(volume_to_uptake)
    
    robot.move(z=z_near_bottom)
    
    
    tube_bottom_is_calibrated = sample._settingPresent('tube_bottom_z')
    not_ignoring_calibration = not ignore_calibration
    if tube_bottom_is_calibrated and not_ignoring_calibration:
        z_lowest = sample.getZBottom()
    else:
        z_lowest = robot.moveDownUntilPress(step=0.1, threshold=100)
    
    liquid_uptake_low_volume_bottom_offset = robot.getLiquidUptakeLowVolBottomOffset()
    z_start_uptake = z_lowest - liquid_uptake_low_volume_bottom_offset
    
    # Moving robot to the pipetting position
    robot.move(z=z_start_uptake)
    # Starting to uptake
    robot.movePipetteToVolume(extra_vol)
    time.sleep(T) # Waiting for the most liquid to uptake
    
    # Moving slightly up
    robot.moveAxisDelta('Z', -liquid_uptake_low_volume_bottom_offset)
    time.sleep(T)
    # Doing a circle
    vol_per_step = extra_vol/4.0
    stepOfACircle(robot, 'X', R, T, liquid_uptake_low_volume_bottom_offset, extra_vol-vol_per_step)
    stepOfACircle(robot, 'X', -R, T, liquid_uptake_low_volume_bottom_offset, extra_vol-2*vol_per_step)
    stepOfACircle(robot, 'Y', R, T, liquid_uptake_low_volume_bottom_offset, extra_vol-3*vol_per_step)
    stepOfACircle(robot, 'Y', -R, T, liquid_uptake_low_volume_bottom_offset, 0)
    
    sample.setVolume(0)  # All liquid must have been uptaken
    
    return True

In [91]:
ber.pickUpNextTip()

(1, 2)

In [92]:
ber.move(z=50)

In [135]:
uptakeAllLiquid(ber, sample1, ignore_calibration=False)

True

In [136]:
ber.move(z=50)

In [137]:
ber.dispenseLiquid(waste, 200)

In [138]:
ber.move(z=50)

In [139]:
ber.dumpTipToWaste()

In [95]:
sample1.getVolume()

0

In [105]:
ber.move(z=50)

In [131]:
ber.uptakeLiquid(water, 130)

In [132]:
ber.move(z=50)

In [133]:
ber.dispenseLiquid(sample1, 150)

In [134]:
ber.move(z=50)

In [50]:
sample1.getCloseToBottomVol()

50

In [53]:
sample1.calcAbsLiquidLevelFromVol(50, added_length=ber._calcExtraLength())

133.825

In [52]:
ber.tip_attached = True

In [49]:
ber.moveToSample?

In [112]:
ber.pickUpNextTip()
ber.move(z=50)

In [22]:
ber.uptakeLiquid(sample1, 30)

In [23]:
ber.move(z=50)

In [41]:
ber.dispenseLiquid(sample1, 50, blow_extra=True)

In [39]:
ber.uptakeLiquid(sample1, 30, ignore_bottom_z_calibration=True)

In [42]:
ber.move(z=50)

In [31]:
ber.movePipetteToVolume(30)

In [34]:
ber.move(z=sample1.getZBottom())

In [35]:
ber.moveDownUntilPress(step=0.1, threshold=100)

136.525

In [36]:
ber.movePipetteToVolume(0)

In [37]:
ber.moveAxisDelta('Z', -0.1)

In [43]:

ber.dumpTipToWaste()

In [8]:
ber.data

{'magnets_away_angle': 5.2,
 'magnets_near_tube_angle': 11.2,
 'stair_finding_step_list': [1, 0.2],
 'stair_finding_z_increment': 0.1,
 'stair_finding_z_retract_after_trigger': -1,
 'stair_finding_z_max_travel': 3,
 'stair_finding_z_load_threshold': 500,
 'z_max': 180,
 'x_max': 189,
 'y_max': 322,
 'added_tip_length': 41.6,
 'volume_to_position_slope': -0.15859245180518214,
 'volume_to_position_intercept': -0.958195131933648,
 'pipetting_delay': 0.2,
 'DNAsize_to_Vbeads': {'a': 0.499325349, 'b': -9.91043764, 'c': 25758.5836},
 'tip_drop_servo_up_angle': 2.5,
 'tip_drop_servo_down_angle': 7.3,
 'speed_XY': 50000,
 'speed_Z': 30000,
 'speed_pipette': 2500,
 'plunger_movement_when_dumping_tip': 35,
 'maximum_plunger_movement_coordinate': 35,
 'liquid_uptake_low_volume_bottom_offset': 0.2}

In [18]:
sample1.getZBottom()

136.425

In [19]:
ber.move(z=sample1.getZBottom())

In [20]:
ber.moveDownUntilPress(step=0.2, threshold=200)

137.025

In [22]:
sample1.setZBottom(137.025)

In [23]:
sample1.getZBottom()

137.025

# Circle experiment

In [7]:
ber.movePipetteToVolume(100)

In [12]:
ber.writeAndWaitCartesian("G2 I10")

'ok\n'

In [13]:
ber.moveToSample(waste)

In [38]:
ber.writeAndWaitCartesian("G0 A0 F1000 G2 I2 F200")

'ok\n'

In [35]:
ber.moveAxisDelta('Z', -0.2)

In [31]:
ber.moveAxisDelta('X', -2)

In [37]:
ber.data

{'magnets_away_angle': 5.2,
 'magnets_near_tube_angle': 11.2,
 'stair_finding_step_list': [1, 0.2],
 'stair_finding_z_increment': 0.1,
 'stair_finding_z_retract_after_trigger': -1,
 'stair_finding_z_max_travel': 3,
 'stair_finding_z_load_threshold': 500,
 'z_max': 180,
 'x_max': 189,
 'y_max': 322,
 'added_tip_length': 41.6,
 'volume_to_position_slope': -0.15859245180518214,
 'volume_to_position_intercept': -0.958195131933648,
 'pipetting_delay': 0.2,
 'DNAsize_to_Vbeads': {'a': 0.499325349, 'b': -9.91043764, 'c': 25758.5836},
 'tip_drop_servo_up_angle': 2.5,
 'tip_drop_servo_down_angle': 7.3,
 'speed_XY': 50000,
 'speed_Z': 30000,
 'speed_pipette': 2500,
 'plunger_movement_when_dumping_tip': 35,
 'maximum_plunger_movement_coordinate': 35,
 'liquid_uptake_low_volume_bottom_offset': 0.2}

In [39]:
ber.move(z=50)

In [41]:
ber.movePipetteToVolume(0)

# A vs V calibration

In [4]:
ber.home()

In [8]:
# Will refill from here
water = bl.createSample('25ml', 'water', ber.reagents_rack, 0, 0, 25000)
# Will dump to here
waste = bl.createSample('25ml', 'liquid_waste', ber.reagents_rack, 0, 1, 0)
# Sample to uptake from. Starts empty
sample1 = bl.createSample('eppendorf', 's1', ber.samples_rack, 1, 0, 0)

In [9]:
ber.pickUpNextTip()

(1, 4)

In [265]:
a = 34.0

In [266]:
ber.move(z=50)

In [267]:
ber.moveAxisDelta('A', 1)

In [268]:
ber.getPosition('A')

0.9582

In [269]:
ber.moveToSample(water)

In [270]:
ber.moveAxisDelta('A', a)

In [271]:
ber.moveAxisDelta('A', -1)

In [272]:
ber.getPosition('A')

33.9582

In [273]:
z_uptake = water.calcNormalPipettingZ(a, 1.0, added_length=ber._calcExtraLength())

In [274]:
z_uptake

98.80950000000001

In [275]:
ber.move(z=z_uptake)

In [276]:
ber.moveAxisDelta('A', -a)

In [277]:
ber.move(z=50)

In [278]:
ber.getPosition('A')

-0.0418

In [279]:
ber.moveToSample(sample1)

In [280]:
ber.moveAxisDelta('A', 35)

In [281]:
ber.moveAxisDelta('A', -35)

In [282]:
ber.move(z=50)

In [28]:
uptakeAllLiquid(ber, sample1)

True

In [6]:
ber.move(z=50)

In [30]:
ber.dispenseLiquid(waste, 200)

In [31]:
ber.move(z=50)

In [33]:
ber.dumpTipToWaste()

In [284]:
ber.data

{'magnets_away_angle': 5.2,
 'magnets_near_tube_angle': 11.2,
 'stair_finding_step_list': [1, 0.2],
 'stair_finding_z_increment': 0.1,
 'stair_finding_z_retract_after_trigger': -1,
 'stair_finding_z_max_travel': 3,
 'stair_finding_z_load_threshold': 500,
 'z_max': 180,
 'x_max': 189,
 'y_max': 322,
 'added_tip_length': 41.6,
 'volume_to_position_slope': -0.15859245180518214,
 'volume_to_position_intercept': -0.958195131933648,
 'pipetting_delay': 0.2,
 'DNAsize_to_Vbeads': {'a': 0.499325349, 'b': -9.91043764, 'c': 25758.5836},
 'tip_drop_servo_up_angle': 2.5,
 'tip_drop_servo_down_angle': 7.3,
 'speed_XY': 50000,
 'speed_Z': 30000,
 'speed_pipette': 2500,
 'plunger_movement_when_dumping_tip': 35,
 'maximum_plunger_movement_coordinate': 35,
 'liquid_uptake_low_volume_bottom_offset': 0.2}

# Using functions integrated to the bernielib

In [5]:
ber.pickUpNextTip()

(1, 9)

In [6]:
ber.move(z=50)

In [9]:
ber.uptakeAllLiquid(sample1)

True

In [9]:
ber.move(z=50)

In [10]:
ber.getPipetteDelay(sample1)

0.2

In [12]:
sample1.stype.data

{'z_above_racks_dict': {'samples': 11.5},
 'inner_diameter': 9.15,
 'depth_to_vol_dict': {'0': 38.45,
  '100': 32,
  '200': 28,
  '300': 26,
  '400': 24,
  '500': 22,
  '700': 18,
  '1000': 13,
  '1500': 5.8,
  '1700': 3},
 'extra_immersion_volume': 200,
 'close_to_bottom_volume': 50,
 'low_vol_uptake_single_step': 0.5,
 'low_vol_uptake_number_of_steps': 3,
 'low_vol_uptake_delay_between_steps': 1}

In [11]:
ber.uptakeLiquid(sample1, 100)

In [12]:
ber.getPosition('a')

0.3457

## Testing uptakeLiquid with bigger volume

In [4]:
# Will refill from here
water = bl.createSample('25ml', 'water', ber.reagents_rack, 0, 0, 15000)
# Will dump to here
waste = bl.createSample('25ml', 'liquid_waste', ber.reagents_rack, 0, 1, 0)
# Sample to uptake from. Starts empty
sample1 = bl.createSample('eppendorf', 's1', ber.samples_rack, 1, 0, 1000)

In [5]:
ber.pickUpNextTip()

(1, 11)

In [6]:
ber.move(z=50)

In [7]:
ber.uptakeLiquid(sample1, 200)

In [9]:
sample1.getCloseToBottomVol()

50

In [10]:
sample1.getVolume()

800

In [11]:
sample1.getExtraImmersionVol()

200

In [26]:
tube_bottom_is_calibrated = False
ignore_calibration = False
v_insert_override = 1
sample_has_low_volume = True
((not tube_bottom_is_calibrated) or ignore_calibration) and ((v_insert_override is None) and (sample_has_low_volume))

False

## Testing uptakeLiquid for calibration

In [7]:
# Will refill from here
water = bl.createSample('25ml', 'water', ber.reagents_rack, 0, 0, 25000)
# Will dump to here
waste = bl.createSample('25ml', 'liquid_waste', ber.reagents_rack, 0, 1, 0)
# Sample to uptake from. Starts empty
sample1 = bl.createSample('eppendorf', 's1', ber.samples_rack, 1, 0, 1000)

In [8]:
ber.pickUpNextTip()
ber.move(z=50)

In [18]:
ber.uptakeLiquid(water, 200)
ber.move(z=50)

In [19]:
ber.dispenseLiquid(sample1, 200, blow_extra=True)
ber.move(z=50)

In [13]:
1026-995

31

In [17]:
1086-988

98

In [20]:
1183-990

193

In [21]:
sample1.stype.data

{'z_above_racks_dict': {'samples': 11.5},
 'inner_diameter': 9.15,
 'depth_to_vol_dict': {'0': 38.45,
  '100': 32,
  '200': 28,
  '300': 26,
  '400': 24,
  '500': 22,
  '700': 18,
  '1000': 13,
  '1500': 5.8,
  '1700': 3},
 'extra_immersion_volume': 200,
 'close_to_bottom_volume': 50,
 'low_vol_uptake_single_step': 0.5,
 'low_vol_uptake_number_of_steps': 3,
 'low_vol_uptake_delay_between_steps': 1}

## Mix scripts

In [5]:
sample1.stype.name

'eppendorf'

In [6]:
sample1.stype.getMixScript()

Error: setting  mix_script_file_path  is not specified.
Use _setSetting('setting_name', value) to specify it.
Alternatively, do self.data['setting_name']=value to set it temporary.


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [7]:
sample1.stype.setMixScriptFilePath('mixing_pattern_eppendorf.csv')

In [8]:
sample1.stype.getMixScript()

,Height,H_relative_to,Plunger_pos,dx,dy,min_vol_condition,max_vol_condition,delay,comment
0,0,top,down,0,0,0,1700,0.5,Initially lowering the plunger
1,1,bottom,up,0,0,0,250,0.5,mixing in place
2,1,bottom,down,0,0,0,250,0.5,mixing in place
3,1,bottom,up,0,0,0,250,0.5,mixing in place
4,1,bottom,down,0,0,0,250,0.5,mixing in place
5,1,bottom,up,0,0,0,250,0.5,mixing in place
6,1,bottom,down,0,0,0,250,0.5,mixing in place
7,1,bottom,up,0,0,0,1700,0.5,"uptaking from bottom, releasing to the top"
8,5,surface,down,0,0,0,1700,0.5,"uptaking from bottom, releasing to the top"
9,1,bottom,up,0,0,0,1700,0.5,"uptaking from bottom, releasing to the top"


In [9]:
beads1 = bl.createSample('5ml', 'beads', ber.reagents_rack, 0, 4, 0)

In [10]:
beads1.stype.setMixScriptFilePath('mixing_pattern_5mL.csv')

# Mix by script

In [11]:
beads1 = bl.createSample('5ml', 'beads', ber.reagents_rack, 0, 4, 1000)

In [12]:
ber.pickUpNextTip()
ber.move(z=50)

In [13]:
ber.mixByScript(beads1, beads1.stype.getMixScript())

In [14]:
ber.move(z=50)

In [15]:
ber.dumpTipToWaste()
ber.move(z=50)